In [ ]:
!pip install kornia --no-index --find-links=file:///kaggle/input/k/oldufo/imc2022-dependencies/pip/kornia/ --upgrade 
!pip install kornia_moons --no-index --find-links=file:///kaggle/input/k/oldufo/imc2022-dependencies/pip/kornia_moons/ --no-deps  --upgrade
!cp -r ../input/imutils/imutils-0.5.3/ /
!pip install /imutils-0.5.3/
print('Done!')

# Imports

In [ ]:
import matplotlib.pyplot as plt
import cv2
import kornia as K
import kornia.feature as KF
import numpy as np
import pandas as pd
import torch
import os
import gc
from kornia_moons.feature import *
from tqdm.notebook import tqdm
import kornia.augmentation as KA

if not torch.cuda.is_available():
    print('You may want to enable the GPU switch?')

# Load data

In [ ]:
DEVICE = 'cuda:0'

In [ ]:
num_features = 8000


class KeyNetAffNetHardNet(KF.LocalFeature):
    """Convenience module, which implements KeyNet detector + AffNet + HardNet descriptor."""
    def __init__(self,
                 num_features: int = 5000,
                 upright: bool = True,
                 device: torch.device = torch.device('cuda')):
        ori_module = KF.PassLAF()
        detector = KF.KeyNetDetector(False,
                                  ori_module=ori_module,
                                  aff_module=KF.LAFAffNetShapeEstimator(False).eval()).to(device)
        detector.model.load_state_dict(torch.load('/kaggle/input/k/oldufo/imc2022-dependencies/pretrained/keynet_pytorch.pth')['state_dict'])
        detector.aff.load_state_dict(torch.load('/kaggle/input/k/oldufo/imc2022-dependencies/pretrained/AffNet.pth')['state_dict'])
        descriptor = KF.LAFDescriptor(KF.HardNet8(False),
                                   patch_size=32,
                                   grayscale_descriptor=True).to(device)
        descriptor.descriptor.load_state_dict(torch.load('/kaggle/input/k/oldufo/imc2022-dependencies/pretrained/hardnet8v2.pt'))
        super().__init__(detector, descriptor)
        
keynet_affnet_hardnet8 = KeyNetAffNetHardNet(num_features).eval()

In [ ]:
train_path = '../input/image-matching-challenge-2022/test_images'

def load_torch_image(fname):
    img = cv2.imread(fname)
    #img = resize_keep_ratio(img)
    img = K.image_to_tensor(img, False).float() /255.
    img = K.color.bgr_to_rgb(img)
    return img

def get_images_filenames(scence):
    path = train_path + '/' + scence 
    img_filename_list = os.listdir(path)
    return img_filename_list, path

def generate_dict(img1_filename, img2_filename):
    img1 = load_torch_image(img1_filename)
    img2 = load_torch_image(img2_filename)
    
    input_dict = {"image0": K.color.rgb_to_grayscale(img1).to(DEVICE), # LofTR works on grayscale images only 
                  "image1": K.color.rgb_to_grayscale(img2).to(DEVICE)}
    
    return input_dict

def FlattenMatrix(M, num_digits=8):
    '''Convenience function to write CSV files.'''
    
    return ' '.join([f'{v:.{num_digits}e}' for v in M.flatten()])

In [ ]:
def load_image(fname):
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = K.image_to_tensor(img, False).float() /255.
    img = K.color.rgb_to_grayscale(img).to(DEVICE)
    return img

* **lafs:** Detected local affine frames with shape(1, 2048, 2, 3).
* **resps** Response function values for corresponding lafs with shape(1, 2048).
* **descriptor** ocal descriptors of shape(1, 2048, 128) where 128 is descriptor size.

In [ ]:
import csv

src = '/kaggle/input/image-matching-challenge-2022/'

test_samples = []
with open(f'{src}/test.csv') as f:
    reader = csv.reader(f, delimiter=',')
    for i, row in enumerate(reader):
        # Skip header.
        if i == 0:
            continue
        test_samples += [row]

# Generate Fundamental_Matrix

In [ ]:
num_features = 5000
matcher = KF.DescriptorMatcher('snn', 0.99)

# Compute this many samples, and fill the rest with random values, to generate a quick submission and check it works without waiting for a full run. Set to -1 to use all samples.
# how_many_to_fill = 500
how_many_to_fill = -1

F_dict = {}
for i, row in enumerate(test_samples):
    sample_id, batch_id, image_1_id, image_2_id = row
    
    if how_many_to_fill >= 0 and i >= how_many_to_fill:
        F_dict[sample_id] = np.random.rand(3, 3)
        continue

    # Extract features.
    with torch.no_grad():
        img1 = load_image(f'{src}/test_images/{batch_id}/{image_1_id}.png')
        img2 = load_image(f'{src}/test_images/{batch_id}/{image_2_id}.png')
        lafs1, resps1, descriptors_1 = keynet_affnet_hardnet8(img1)
        lafs2, resps2, descriptors_2 = keynet_affnet_hardnet8(img2)
        
        if descriptors_1.size(1) == 0 or descriptors_2.size(1) == 0:
            F_dict[sample_id] = np.zeros((3, 3))
            continue

        dists, idxs  = matcher(descriptors_1[0], descriptors_2[0])
        cur_kp1 = KF.get_laf_center(lafs1).detach().cpu().numpy().reshape(-1, 2)
        cur_kp2 = KF.get_laf_center(lafs2).detach().cpu().numpy().reshape(-1, 2)
        match_idxs = idxs.detach().cpu().numpy()

    # Make sure we do not trigger an exception here.
    if len(match_idxs) > 8:
        F, inlier_mask = cv2.findFundamentalMat(cur_kp1[match_idxs[:, 0]], cur_kp2[match_idxs[:, 1]],
                                                cv2.USAC_MAGSAC,
                                                ransacReprojThreshold=0.5,
                                                confidence=0.99999,
                                                maxIters=10000)
        assert F.shape == (3, 3), 'Malformed F?'
        F_dict[sample_id] = F
    else:
        F_dict[sample_id] = np.zeros((3, 3))
        continue
    gc.collect()

# Make an submission

In [ ]:
with open('submission.csv', 'w') as f:
    f.write('sample_id,fundamental_matrix\n')
    for sample_id, F in F_dict.items():
        f.write(f'{sample_id},{FlattenMatrix(F)}\n')